In [8]:
from itertools import combinations_with_replacement

In [1]:
from utils import *

In [2]:
# train_data = load_data("data/text1-Pnar15_LathadlabotStory3.txt")
test_data = load_data("data/text2-Pnar05_DaloiofRaliang.txt")
train_data = load_data("data/full_text.txt")

for data in test_data:
    train_data.remove(data)


In [4]:
test_data

[{'tx': 'nga toh u-chwas lyngdoh u-daloi jong ka-elaka raliang. ka-jingmut daloi toh u-nongshynshar hapoh ka-elaka',
  'mb': 'nga toh u= chwas lyngdoh u= daloi jong ka= elaka raliang ka= jing- mut daloi toh u= nong- synchar ha- poh ka= elaka',
  'ph': 'ŋa tɔʔ ˀu= ʧwas lŋ̩dɔʔ ˀu= daloj ʤɔŋ ka= ˀɛlaka raliaŋ ka= ʤiŋ- mut daloj tɔʔ ˀu= nɔŋ- sn̩ʧar ha- pɔʔ ka= ˀɛlaka',
  'ge': '1sg.top be m= chawas lyngdoh m= daloi gen f= village,township raliang f= ab.nmz- means daloi be m= ag.nmz- rule loc- under,in f= village,township',
  'ps': 'pn v clitic= name n clitic= n khasi clitic= n n clitic= nmz- v n v clitic= nmz- v case- loc clitic= n',
  'ft': "i am chwas lyngdoh, the daloi of the raliang elaka. the daloi's meaning/purpose is (as) the ruler in the elaka"},
 {'tx': 'bad u-khleih yong ka-niam ka-rukom yong ka-dustur ka-riti hapoh ka-elaka ka yong i',
  'mb': 'bad u= khleih yong ka= niam ka= rukom yong ka= dustur ka= riti ha- poh ka= elaka ka yong i',
  'ph': 'baːt ˀu= kʰleʲʔ jɔŋ ka= niam ka= r

In [8]:
for data in test_data:
    train_data.remove(data)

len(train_data)

2610

In [20]:
def search_gloss_block(block, gloss_query_list, preceding_pos=[], gloss_pos=[], post_pos=[]):
    
    # Split gloss and pos into lists
    gloss = block[GLOSS].split(" ")
    pos = block[POS].split(" ")
    # Check if the input variable gloss_pos is valid
    check_gloss_pos = True if len(gloss_pos) == len(gloss_query_list) else False

    candidates = []
    # Iterate through gloss by index
    for i in range(len(gloss)):
        # Perform window search to find matching queries
        gloss_window = gloss[i:i+len(gloss_query_list)]
        pos_window = pos[i:i+len(gloss_query_list)]

        # Check if gloss matches
        if gloss_window == gloss_query_list:
            # Check if pos matches
            if check_gloss_pos and pos_window != gloss_pos:
                continue
            else:
                candidates.append(i)


    # return False if no candidates found
    if len(candidates) == 0:
        return False
    # return true if candidates found and user does not want to check preceding/post pos
    elif len(preceding_pos) == 0 and len(post_pos) == 0:
        return True

    # Iterate through each candidate index
    for idx in candidates:
        # Get the starting and ending indexes to match
        start_idx = idx - len(preceding_pos)
        end_idx = idx + len(gloss_query_list) + len(post_pos)

        # Skip if indexes are out of range
        if start_idx < 0 or end_idx > len(pos):
            continue
        
        # Check if preceding and post pos are matching
        if pos[start_idx:idx] == preceding_pos and pos[idx+len(gloss_query_list): end_idx] == post_pos:
            return True
    
    return False


def search_gloss_data(data, gloss_query, preceding_pos=[], gloss_pos=[], post_pos=[]):
    results = []
    # Preprocess query 
    gloss_query_list = gloss_query.lower().strip().split(" ")

    for idx, block in enumerate(data):
        if search_gloss_block(block, gloss_query_list, preceding_pos, gloss_pos, post_pos):
            results.append(idx)

    return results


def search_text_block(block, text_query):
    text = block[TEXT]
    if text_query in text:
        return True
    else:
        return False


def search_text_data(data, text_query):
    results = []

    text_query = text_query.lower().strip()

    for idx, block in enumerate(data):
        if search_text_block(block, text_query):
            results.append(idx)
    
    return results


def combine_search_results(results):
    intersections = set.intersection(*[set(list) for list in results])
    return sorted(list(intersections))


def format_block(data, ids):
    s = ""
    for id in ids:
        block = data[id]
        s += "\n".join([f"{tag}: {block[tag]}" for tag in block.keys()])
        s += "\n" + "---"*40 + "\n"

    return s

In [23]:
def get_pos_tags(data):
    # Get all unique POS tags in the data
    tags = []
    for block in data:
        tags += block[POS].split(" ")
    tags = list(set(tags))
    return tags

In [21]:

r = search_data(data, [], preceding_pos=["n"], gloss_pos=["n"], post_pos=[])
print(format_block(data, r))

tx: nga retired o ha u-snæm khatkhynde spah sanphaw soo
mb: nga retired o ha u= snæm khatkhynde spah sanphaw soo
ph: ŋa ritait ˀɔ ha ˀu= snɛm kʰatkʰn̩de spaʔ sanpʰaw so
ge: 1sg.top retired 1sg.nom loc m= year nineteen hundred fifty four
ps: pn english pn case clitic= n num num num num
ft: i retired from my service in the year 1954
------------------------------------------------------------------------------------------------------------------------
tx: ha u-snæm khatkhynde spah sanphaw san, lai shillong
mb: ha u= snæm khatkhynde spah sanphaw san lai shillong
ph: ha ˀu= snɛm kʰatkʰn̩de spaʔ sanpʰaw san laj ʃl̩lɔŋ
ge: loc m= year nineteen hundred fifty five go shillong
ps: case clitic= n num num num num v name
ft: in the year 1955, i went to shillong
------------------------------------------------------------------------------------------------------------------------
tx: ki pyn-sum kule
mb: ki pyn- sum kule
ph: ki pn̩- sum kule
ge: 3pl caus- bathe horse
ps: pn prefix- n n
ft: i went t